 Đọc 3 file dữ liệu và khảo sát lỗi

In [1]:
import pandas as pd
import numpy as np

path_info = "../data_raw/stock_info.csv"
path_price = "../data_raw/stock_price.csv"
path_trans = "../data_raw/transactions.csv"

df_info = pd.read_csv(path_info)
df_price = pd.read_csv(path_price)
df_trans = pd.read_csv(path_trans)

df_info.head(), df_price.head(), df_trans.head()


#Khảo sát lỗi dữ liệu
print("INFO:")
print(df_info.info())
print(df_info.isna().sum())

print("\nPRICE:")
print(df_price.info())
print(df_price.isna().sum())

print("\nTRANSACTIONS:")
print(df_trans.info())
print(df_trans.isna().sum())



INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ticker    10 non-null     object
 1   exchange  10 non-null     object
 2   sector    10 non-null     object
dtypes: object(3)
memory usage: 372.0+ bytes
None
ticker      0
exchange    0
sector      0
dtype: int64

PRICE:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ticker       400 non-null    object
 1   date         400 non-null    object
 2   close_price  400 non-null    object
dtypes: object(3)
memory usage: 9.5+ KB
None
ticker         0
date           0
close_price    0
dtype: int64

TRANSACTIONS:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------  

 Chuẩn hóa mã cổ phiếu sang dạng viết hoa

In [2]:
df_info['ticker'] = df_info['ticker'].astype(str).str.strip().str.upper()
df_price['ticker'] = df_price['ticker'].astype(str).str.strip().str.upper()
df_trans['ticker'] = df_trans['ticker'].astype(str).str.strip().str.upper()
print(df_info['ticker'].head(), "\n")

0    VIC
1    VHM
2    VNM
3    HPG
4    FPT
Name: ticker, dtype: object 



 Xử lý ngày tháng sai định dạng

In [3]:
df_price['date'] = pd.to_datetime(df_price['date'], errors='coerce', dayfirst=True)
print(df_price['date'].head(), "\n") 


0   2024-01-01
1   2024-01-02
2   2024-01-03
3          NaT
4   2024-01-05
Name: date, dtype: datetime64[ns] 



 Làm sạch close_price và volume

In [4]:
# Clean CLOSE_PRICE

df_price['close_price'] = (
    df_price['close_price']
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.replace(" ", "", regex=False)
)

df_price['close_price'] = pd.to_numeric(df_price['close_price'], errors='coerce')
print(df_price['close_price'].head(), "\n")

# 6. Clean VOLUME

df_trans['volume'] = (
    df_trans['volume']
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.replace(" ", "", regex=False)
)

df_trans['volume'] = pd.to_numeric(df_trans['volume'], errors='coerce')

# Loại âm
before = len(df_trans)
df_trans = df_trans[df_trans['volume'] >= 0]
after = len(df_trans)

print(f" Đã xoá {before - after} dòng volume âm")
print(df_trans['volume'].head(), "\n")


0    52000.0
1        NaN
2    52000.0
3        NaN
4    50000.0
Name: close_price, dtype: float64 

 Đã xoá 87 dòng volume âm
0    10000.0
1    20000.0
2    10000.0
3    20000.0
4    20000.0
Name: volume, dtype: float64 



 Chuẩn hóa thông tin sàn và ngành

In [5]:
# Exchange
df_info['exchange'] = (
    df_info['exchange']
    .astype(str)
    .str.strip()
    .str.upper()
    .replace({
        "UPCOM": "UPCOM",
        "UP COM": "UPCOM",
        "HNX ": "HNX",
        " HOSE": "HOSE"
    })
)
print(df_info['exchange'].unique(), "\n")

# Sector
df_info['sector'] = (
    df_info['sector']
    .astype(str)
    .str.strip()
    .str.title()    # Viết Hoa Chữ Cái Đầu
)
print(df_info['exchange'].unique(), "\n")


['UPCOM' 'HOSE' 'HNX'] 

['UPCOM' 'HOSE' 'HNX'] 



Kiểm tra dữ liệu

In [6]:
print(df_info.head(), "\n")
print(df_price.head(), "\n")
print(df_trans.head(), "\n")

  ticker exchange     sector
0    VIC    UPCOM  Tai Chinh
1    VHM    UPCOM  Công Nghệ
2    VNM    UPCOM     Bán Lẻ
3    HPG     HOSE  Tai Chinh
4    FPT    UPCOM     Bán Lẻ 

  ticker       date  close_price
0    VIC 2024-01-01      52000.0
1    VIC 2024-01-02          NaN
2    VIC 2024-01-03      52000.0
3    VIC        NaT          NaN
4    VIC 2024-01-05      50000.0 

  ticker   volume trade_type
0    VIC  10000.0      B-U-Y
1    VIC  20000.0      B-U-Y
2    VIC  10000.0        buy
3    VIC  20000.0       SELL
4    VIC  20000.0       sell 



 Xuất các file sạch 

In [7]:
df_info.to_csv("../data_clean/stock_info_clean.csv", index=False)
df_price.to_csv("../data_clean/stock_price_clean.csv", index=False)
df_trans.to_csv("../data_clean/transactions_clean.csv", index=False)

print(" Đã xuất file sạch vào thư mục data_clean")


 Đã xuất file sạch vào thư mục data_clean
